In [1]:
import re, string 
import pandas as pd 
from time import time  
from collections import defaultdict
import spacy
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from transformers import BertModel, BertConfig, BertTokenizer, PreTrainedTokenizer
import csv

In [2]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower().strip()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    #removes unicodes left in text so model does not learn unicodes
    text = re.sub('^\\\\u[\d\D]{4}|-|σ|→|\\\\xad', '', text)
    # Remove a sentence if it is only one word long
    if len(text) > 2:
        return ' '.join(word for word in text.split() if word not in STOPWORDS)
    return 


In [3]:
def tokenizer(text):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokens = tokenizer.tokenize(str(text), add_special_tokens=True)
    return tokens

In [4]:
nlp = spacy.load('en_core_web_sm')

def lemmatizer(text):
    if text == None:
        print("Excuse me but there is an unexpected None value from cleaning the text!")
        return ""
    sent = []
    tokens = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

In [5]:
def word_2_vec(filepath, w2vmodel, tokenize=False, last_model=None):
    global vocab
    #Creates a pandas dataframe for the text data from a json file
    df = pd.read_json(filepath)
    
    #Adds the column label text
    df.columns = ['text']
    
    #Text clean function is applied to the pandas dataframe
    df_clean = pd.DataFrame(df.text.apply(lambda x: clean_text(x)))
    
    #filters out all the None values in the cleaned dataset
    #loc combines the operations in brackets into one single operation to avoid chaining indexes operations together
    #copy explicitly tells pandas to make copy when creating master_of_none
    #this is so later on only the copy is modified and there is no confusion between the copy and the original
    master_of_none = df_clean.loc[df_clean.text.notnull()].copy()
    
    #Lemmatizer function is applied to cleaned text with the none values removed
    master_of_none["text_lemmatized"] =  master_of_none.apply(lambda x: lemmatizer(x['text']), axis=1)
    
    master_of_none['text_lemmatize_clean'] = master_of_none['text_lemmatized'].str.replace('-PRON-', '')

    sentences = [row.split() for row in master_of_none['text_lemmatize_clean']]
    word_freq = defaultdict(int)
    for sent in sentences:
    #tokens = tokenizer(sent)
        for i in sent:
            word_freq[i] += 1
    len(word_freq)

    if last_model is not None:
        
        #Previously updated model is loaded
        w2v_model = Word2Vec.load(last_model)  
        
        #model vocabulary is updated
        w2v_model.build_vocab(sentences, update=True)

    
    else:
        # min_count: minimum number of occurrences of a word in the corpus to be included in the model.
        # window: the maximum distance between the current and predicted word within a sentence.
        # size: the dimensionality of the feature vectors
        # workers: the number of cores your computer has
        w2v_model = Word2Vec(min_count=50,
                             window=5,
                             size=400,
                             workers=2)
        #the new model's vocabulary is built 
        w2v_model.build_vocab(sentences)

    # train word vectors
    #returns the number of words in the vocab and the number of words in the corpus
    w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_model.epochs)
    
    #either the new or updated version of the w2v model is saved
    w2v_model.save(w2vmodel)
    return print("Training complete!")

In [6]:

first_word = ['alcohol', 'ketone', 'alkene', 'carbon', 'proton', 'polymer', 'acid', 'oxidize', 'anion', 'electrophile', 'polar', 'positive', 'mechanism', 'resonance', 'synthesis', 'isomer', 'heat', 'aromatic', ]
second_word = ['hydroxyl', 'carbonyl', 'alkyne', 'nitrogen', 'hydrogen', 'chain', 'base', 'reduce', 'cation', 'nucleophile', 'nonpolar', 'negative', 'atom', 'solvent', 'electron', 'reaction', 'bond', 'equilibrium']


def cosine_sim_saver(w2vmodel, first_word, second_word):
    
    global w2v_data
    
    w2v_model = Word2Vec.load(w2vmodel)
    
    w2v_data = []
    
    
    for word1, word2 in zip(first_word, second_word):
        try:
        #synonym, antonym, or neutral pairs
            cos_sim = w2v_model.wv.similarity(word1, word2)
            w2v_data.append(cos_sim)
        except KeyError:
            cos_sim = 0 
            w2v_data.append(cos_sim)
            print("One word was not in the vocabulary")
   
    try:
        top10 = w2v_model.wv.most_similar(positive=['carbon'])
        w2v_data.append(top10)
    except KeyError:
        top10 = 0
        print("carbon is not in the vocabulary")
    
    
    
    return print("Data collection complete!")
    

In [7]:
my_headers = ['model_name', 'alcohol + hydroxyl', 'ketone + carbonyl', 'alkene + alkyyne', 'carbon + nitrogen', 'proton + hydrogen', 'polymer + chain', 'acid + base', 'oxidize + reduce', 'anion + cation', 'electrophile + nucleophile', 'polar + nonpolar', 'positive + negative', 'mechanism + atom', 'resonance + solvent', 'synthesis + electron', 'isomer + reaction', 'heat + bond', 'aromatic + equilibrium', 'Top 10 Carbon']

def data_saver(excel_file, my_headers, new_file=None):
    if new_file is not None:
        with open(excel_file, 'w') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(my_headers)
            csvwriter.writerow(w2v_data)
    else:
        with open(excel_file, 'a+', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(w2v_data)
    return print("Data saved as CSV")

In [8]:
def tsne_grapher(w2vmodel):
    "Create TSNE model and plot it"
    labels = []
    tokens = []
    model = Word2Vec.load(w2vmodel)

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(18, 18)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()
    return

In [12]:
file1 = 'ChemLibre_JSONS/Basic_Principles_of_Organic_Chemistry_Roberts_and_Caserio.json'
file2 = 'ChemLibre_JSONS/Bruice_Map.json'
file3 = 'ChemLibre_JSONS/Catalytic_Asymmetric_Synthesis_Punniyamurthy.json'
file4 = 'ChemLibre_JSONS/Environmental_Chemistry.json'
file5 = 'ChemLibre_JSONS/How_to_be_a_Successful_Organic_Chemist_Sandtorv.json'
file6 = 'ChemLibre_JSONS/Logic_of_Organic_Synthesis_Rao.json'
file7 = 'ChemLibre_JSONS/Organic_Chemistry_A _Carbonyl_Early_Approach_McMichael.json'
file8 = 'ChemLibre_JSONS/Organic_Chemistry_Lab_Techniques_Nichols.json'
file9 = 'ChemLibre_JSONS/Organic_Chemistry_with_a_Biological_Emphasis_Soderberg.json'
file10 = 'ChemLibre_JSONS/Polymer_Chemistry.json'
file11 = 'ChemLibre_JSONS/Radical_Reactions_of_Carbohydrates_Binkley.json'
file12 = 'ChemLibre_JSONS/Schaller_Polymer.json'
file13 = 'ChemLibre_JSONS/Supplemental_Modules.json'
file14 = 'ChemLibre_JSONS/Wade_Map.json'
file15 = 'Springer_PDF/Brewing_Science_A_Multidisciplinary_Approach_by_Mosher_and_Trantham.json'
file16 = 'Springer_PDF/Advanced_Organic_Chemistry_Part_A_Structure_and_Mechanisms_by_Carey_and_Sundberg.json'
file17 = 'Springer_PDF/Advanced_Organic_Chemistry_Part_B_Reactions_and_Synthesis_by_Carey_and_Sundberg.json'
file18 = 'Springer_PDF/Principles_of_Polymer_Chemistry_by_Ravve.json'
file19 = 'Springer_PDF/Polymer_Synthesis_Theory_and_Practice_by_Braun_Cherdron_Rehahn_Ritter_and_Voit.json'
file20 = 'Springer_PDF/Polymer_Chemistry_by_Koltzsenburg_Maskos_and_Nuyken.json'

In [10]:
model1 = 'robers_and_caserio.model'
model2 = 'bruice.model'
model3 = 'punniyamurthy.model'
model4 = 'environmental.model'
model5 = 'sandtorv.model'
model6 = 'rao.model'
model7 = 'mcmichael.model'
model8 = 'nichols.model'
model9 = 'soderberg.model'
model10 = 'polymer.model'
model11 = 'binkley.model'
model12 = 'schaller.model'
model13 = 'supplemental.model'
model14 = 'wade.model'
model15 = 'mosher_and_trantham.model'
model16 = 'a_carey_and_sundberg.model'
model17 = 'b_carey_and_sundberg.model'
model18 = 'ravve.model'
model19 = 'braun_chedron_rehahn_ritter_and_voit.model'
model20 = 'koltzsenburg_maskos_and_nuyken.model'

In [11]:
word_2_vec(file1, model1)
word_2_vec(file2, model2)
word_2_vec(file3, model3)
word_2_vec(file4, model4)
word_2_vec(file5, model5)
word_2_vec(file6, model6)
word_2_vec(file7, model7)
word_2_vec(file8, model8)
word_2_vec(file9, model9)
word_2_vec(file10, model10)
word_2_vec(file11, model11)
word_2_vec(file12, model12)
word_2_vec(file13, model13)
word_2_vec(file14, model14)


Training complete!
Training complete!
Training complete!
Training complete!
Training complete!
Training complete!
Training complete!
Training complete!
Training complete!
Training complete!
Training complete!
Training complete!
Training complete!
Training complete!


ValueError: Expected object or value

In [13]:
word_2_vec(file15, model15)
word_2_vec(file16, model16)
word_2_vec(file17, model17)
word_2_vec(file18, model18)
word_2_vec(file19, model19)
word_2_vec(file20, model20)

Training complete!
Training complete!
Training complete!
Training complete!
Training complete!
Training complete!


In [32]:
word_2_vec(file18, model18)

Training complete!


In [14]:
cosine_sim_saver(model1, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=True)

One word was not in the vocabulary
One word was not in the vocabulary
Data collection complete!
Data saved as CSV


In [15]:
cosine_sim_saver(model2, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
Data collection complete!
Data saved as CSV


In [16]:
cosine_sim_saver(model3, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
carbon is not in the vocabulary
Data collection complete!
Data saved as CSV


In [17]:
cosine_sim_saver(model4, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
Data collection complete!
Data saved as CSV


In [18]:
cosine_sim_saver(model5, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
carbon is not in the vocabulary
Data collection complete!
Data saved as CSV


In [19]:
cosine_sim_saver(model6, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
carbon is not in the vocabulary
Data collection complete!
Data saved as CSV


In [20]:
cosine_sim_saver(model7, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
Data collection complete!
Data saved as CSV


In [21]:
cosine_sim_saver(model8, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
carbon is not in the vocabulary
Data collection complete!
Data saved as CSV


In [35]:
cosine_sim_saver(model9, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
Data collection complete!


UnicodeEncodeError: 'charmap' codec can't encode character '\u03b1' in position 293: character maps to <undefined>

In [23]:
cosine_sim_saver(model10, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
carbon is not in the vocabulary
Data collection complete!
Data saved as CSV


In [24]:
cosine_sim_saver(model11, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
carbon is not in the vocabulary
Data collection complete!
Data saved as CSV


In [25]:
cosine_sim_saver(model12, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
carbon is not in the vocabulary
Data collection complete!
Data saved as CSV


In [26]:
cosine_sim_saver(model13, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
Data collection complete!
Data saved as CSV


In [27]:
cosine_sim_saver(model14, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

Data collection complete!
Data saved as CSV


In [28]:
cosine_sim_saver(model15, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
Data collection complete!
Data saved as CSV


In [29]:
cosine_sim_saver(model16, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
Data collection complete!
Data saved as CSV


In [30]:
cosine_sim_saver(model17, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
Data collection complete!
Data saved as CSV


In [36]:
cosine_sim_saver(model18, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
Data collection complete!
Data saved as CSV


In [33]:
cosine_sim_saver(model19, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
Data collection complete!
Data saved as CSV


In [34]:
cosine_sim_saver(model20, first_word, second_word)
data_saver('all_texts.csv', my_headers, new_file=False)

One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
One word was not in the vocabulary
Data collection complete!
Data saved as CSV
